# Modelos



## Introducción a los Modelos de Machine Learning
En el análisis de datos, los modelos de Machine Learning nos permiten identificar patrones y hacer predicciones basadas en datos históricos. Uno de los modelos más interpretables y eficientes para problemas de clasificación es el **Árbol de Decisión** (*Decision Tree*). Este modelo divide los datos en diferentes ramas según criterios de segmentación, facilitando la interpretación de los factores que influyen en la toma de decisiones.

## Objetivo del Análisis
Este estudio busca entender los factores que influyen en la compra de **múltiples coches** en comparación con la compra de un solo vehículo. La meta es desarrollar un modelo que permita predecir si un cliente comprará **uno o más de un coche**, y con ello, generar estrategias que incentiven la compra de múltiples unidades.

## Variable Objetivo (*Target*)
La variable de interés en este modelo es **"Mas_1_coche"**, la cual toma dos posibles valores:
- `0`: El cliente compró **un solo coche**.
- `1`: El cliente compró **más de un coche**.

## Variables Predictoras (*Features*)
Para predecir la variable objetivo, utilizaremos un conjunto de variables predictoras que incluyen características del cliente, del vehículo y del historial de compra. Estas variables serán seleccionadas en base a su relevancia para mejorar el rendimiento del modelo.

## Modelo de Árbol de Decisión
Utilizaremos un **Decision Tree Classifier** para modelar el problema. Este método es ideal para entender la relación entre las variables y la compra de vehículos, ya que permite visualizar las reglas de decisión en forma de ramas jerárquicas. Además, exploraremos **hiperparámetros clave** como:
- `max_depth`: Profundidad máxima del árbol.
- `min_samples_split`: Número mínimo de muestras para dividir un nodo.
- `criterion`: Métrica de evaluación para la división (gini o entropía).


## Importación de librerías

In [15]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, roc_auc_score,
    roc_curve, precision_recall_curve, precision_score, recall_score, f1_score
)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import (
    train_test_split, cross_val_score, learning_curve, validation_curve
)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from tabulate import tabulate
from sklearn.model_selection import GridSearchCV


In [16]:
df = pd.read_csv('../data/Propensity_Processed.csv')
df.head()

,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,PROVINCIA,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,0,0,0,0.0,1,0,0,1,1,4,...,1.0,0,3.0,1,2892,0,0,2,18,0
1,0,0,0,0.0,1,0,0,0,1,47,...,1.0,1,0.0,0,1376,7187,0,2,53,0
2,0,0,0,0.0,1,3,0,1,1,30,...,2.0,0,0.0,0,1376,0,1,4,21,3
3,0,0,0,0.0,1,2,0,0,1,32,...,2.0,1,3.0,1,2015,7256,1,4,48,5
4,0,0,0,0.0,1,2,0,0,2,41,...,3.0,0,0.0,0,1818,0,1,3,21,3


In [17]:
# Dividir los datos en características (X) y etiquetas (y)
X = df.drop(['Mas_1_coche', 'Tiempo'], axis=1)
y = df['Mas_1_coche']

# Convertir características categóricas a numéricas con OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
X[X.select_dtypes(include=['object']).columns] = ordinal_encoder.fit_transform(X.select_dtypes(include=['object']))

# Manejar valores nulos (si existen) después de la conversión
X.fillna(0, inplace=True)

# Eliminar la columna 'CODE' si existe
drop_cols = ['CODE', 'Tiempo']
X.drop(columns=[col for col in drop_cols if col in X.columns], inplace=True)

# Obtener información del dfframe
print(df.head())
print(df.info())
print(X.info())
print(y.info())

   PRODUCTO  TIPO_CARROCERIA  COMBUSTIBLE  Potencia  TRANS  FORMA_PAGO  \
0         0                0            0       0.0      1           0   
1         0                0            0       0.0      1           0   
2         0                0            0       0.0      1           3   
3         0                0            0       0.0      1           2   
4         0                0            0       0.0      1           2   

   ESTADO_CIVIL  GENERO  OcupaciOn  PROVINCIA  ...  Zona_Renta  REV_Garantia  \
0             0       1          1          4  ...         1.0             0   
1             0       0          1         47  ...         1.0             1   
2             0       1          1         30  ...         2.0             0   
3             0       0          1         32  ...         2.0             1   
4             0       0          2         41  ...         3.0             0   

   Averia_grave  QUEJA_CAC  COSTE_VENTA  km_anno  Mas_1_coche  Revisiones 

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir y_train a formato numérico
y_train = y_train.astype(int)

# Convertir y_test a formato numérico
y_test = y_test.astype(int)

# Crear el modelo RandomForestClassifier con 100 árboles y una semilla para reproducibilidad
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo con los datos de entrenamiento
rf_model.fit(X_train, y_train)

# Realizar predicciones sobre el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Evaluar el desempeño del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", accuracy)
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

Precisión del modelo: 0.8301756207284368

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.86      0.91      0.89      8279
           1       0.74      0.62      0.67      3280

    accuracy                           0.83     11559
   macro avg       0.80      0.77      0.78     11559
weighted avg       0.82      0.83      0.83     11559



In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200, 300],  # Número de árboles en el bosque
    'max_depth': [5, 10, 20, None],  # Profundidad máxima del árbol
    'min_samples_split': [2, 5, 10],  # Número mínimo de muestras para dividir un nodo
    'min_samples_leaf': [1, 2, 5, 10],  # Número mínimo de muestras en una hoja
    'criterion': ['gini', 'entropy'],  # Función para medir la calidad de la división
    'max_features': ['sqrt', 'log2', None],  # Número de características consideradas en cada división
    'bootstrap': [True, False],  # Si se usa muestreo con reemplazo
    'class_weight': [None, 'balanced'],  # Ponderación de clases
    'min_weight_fraction_leaf': [0.0, 0.01, 0.05],  # Fracción mínima del peso de muestras en una hoja
    'max_leaf_nodes': [None, 10, 20, 50],  # Número máximo de nodos hoja en el árbol
    'warm_start': [False, True]  # Reutilizar árboles previos para acelerar entrenamiento
}


# Instanciar el modelo base
rf_model = RandomForestClassifier(random_state=42)

# Aplicar GridSearchCV para encontrar la mejor combinación de hiperparámetros
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ajustar el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo
best_rf_model = grid_search.best_estimator_

# Evaluar el mejor modelo en el conjunto de prueba
y_pred = best_rf_model.predict(X_test)

# Métricas de evaluación
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo optimizado:", accuracy)
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", grid_search.best_params_)